In [1]:
import predict

In [2]:
import pandas as pd

In [3]:

data = pd.read_csv('/Users/ltorrick/DS/sibur_2021/data/sc2021_train_deals.csv', parse_dates=["month", "date"])

In [4]:
res = predict.predict(data[data.month<"2020-07-01"], pd.Timestamp("2020-07-01"))

In [5]:
res

,material_code,company_code,country,region,manager_code,prediction
0,134,0,Литва,Литва,12261,5.871563
1,197,0,Китай,Китай,16350,8.571716
2,794,2162,Казахстан,Атырауская обл.,10942,25.213905
3,133,0,Китай,Китай,17745,125.074188
4,133,0,Китай,Китай,16079,126.437241
...,...,...,...,...,...,...
936,193,8656,Россия,Белгородская обл.,12444,36.074799
937,142,9499,Россия,Респ. Татарстан,19466,3.651035
938,602,7033,Сербия,Сербия,10942,136.385452
939,890,0,Китай,Китай,13346,164.978485


In [29]:
!ls

FF_w.pt                encoders.pickle        rnn_test_predict.ipynb
RNN_w.pt               hash2ind.pickle        runs
__pycache__            predict.py
encoder_w.pt           rnn.ipynb


In [30]:
!zip -r baseline_submission_lstm_msle_loss.zip FF_w.pt RNN_w.pt encoders.pickle  hash2ind.pickle encoder_w.pt predict.py

  adding: FF_w.pt (deflated 11%)
  adding: RNN_w.pt (deflated 7%)
  adding: encoders.pickle (deflated 45%)
  adding: hash2ind.pickle (deflated 81%)
  adding: encoder_w.pt (deflated 18%)
  adding: predict.py (deflated 72%)


In [6]:
AGG_COLS = ["material_code", "company_code", "country", "region", "manager_code"]

In [18]:
gr = data.groupby(AGG_COLS + ["month"])["volume"].sum().unstack(fill_value=0)
gr = gr.reset_index()[AGG_COLS+[gr.columns[-1]]]
gr.columns= AGG_COLS+['target']

In [20]:
for_test = pd.merge(gr, res, on=AGG_COLS, how='inner')

In [21]:
for_test

,material_code,company_code,country,region,manager_code,target,prediction
0,124,7278,Россия,Респ. Татарстан,17460,0.0,0.952333
1,133,0,Белоруссия,Минская обл.,10942,249.0,43.147625
2,133,0,Белоруссия,Могилевская обл.,10942,0.0,4.649296
3,133,0,Белоруссия,г. Минск,10942,6.0,2.347144
4,133,0,Казахстан,г. Нур-Султан,13301,0.0,4.173047
...,...,...,...,...,...,...,...
936,986,9943,Россия,Смоленская обл.,17460,83.0,9.635116
937,998,0,Россия,Ленинградская обл.,18079,9.0,3.524394
938,998,3380,Россия,Ленинградская обл.,14956,50.0,51.330097
939,998,5410,Россия,г. Санкт-Петербург,14956,0.0,39.763954


In [22]:
from sklearn.metrics import mean_squared_log_error

In [53]:
def calib(pr):
    if pr<3:
        return 0
    if pr >10:
        return pr*10
    return pr

In [54]:
mean_squared_log_error(for_test['target'], for_test['prediction'].apply(calib))

5.376642511990849

In [26]:
for_test.sort_values('target')

,material_code,company_code,country,region,manager_code,target,prediction
0,124,7278,Россия,Респ. Татарстан,17460,0.0,0.952333
453,395,0,Россия,Новосибирская обл.,10642,0.0,11.506877
451,395,0,Россия,Владимирская обл.,18956,0.0,2.675674
449,394,6537,Россия,Московская обл.,10670,0.0,16.859856
443,392,0,Россия,Тюменская обл.,16656,0.0,20.984354
...,...,...,...,...,...,...,...
866,876,8792,Россия,Тульская обл.,13032,2640.0,110.079872
723,602,0,Украина,Украина,18156,2866.0,185.174683
577,486,8656,Россия,Московская обл.,12444,2955.0,192.305984
849,794,6038,Россия,Ростовская обл.,13032,3155.0,187.838425


In [27]:
for_test['prediction'].describe()

count    941.000000
mean      38.099774
std       44.216503
min        0.623031
25%        5.599281
50%       19.080227
75%       55.064930
max      209.858994
Name: prediction, dtype: float64

In [28]:
for_test['target'].describe()

count     941.000000
mean      144.031881
std       352.298575
min         0.000000
25%         0.000000
50%        40.000000
75%       119.000000
max      3490.000000
Name: target, dtype: float64